## Spotify API

In [ ]:
import pandas as pd
import spotipy
import spotipy.util as util
import requests

# Authentication

def ispotify(client_id, secret, username):
    redirect_uri = 'http://localhost:7777/callback'
    scope = 'user-top-read'
    token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=secret,     
                                   redirect_uri=redirect_uri)
    
    
# Request data from spotify
    
    headers = {'Accept': 'application/json',
               'Content-Type': 'application/json',
               'Authorization': f'Bearer ' + token,
              }
    response = requests.get('https://api.spotify.com/v1/me/top/tracks?limit=50', headers=headers)

    try:
        top = response.json() # transform to json
    except:
        top = None

    track = list()
    album = list()
    artist = list()
    track_id = list()
    popularity = list()

    for i in range(0,50):
        track.append(top["items"][i]["name"])
        album.append(top["items"][i]["album"]["name"])
        artist.append(top["items"][i]["artists"][0]["name"])
        track_id.append(top["items"][i]["id"])
        popularity.append(top["items"][i]["popularity"])
        
        
# use track id to request audio feature from Spotify API

    seperator = ','
    ids = seperator.join(track_id)

    sp = spotipy.Spotify(auth=token)
    feature = sp.audio_features(tracks=[ids])

    danceability = list()
    energy = list()
    loudness = list()
    speechiness = list()
    acousticness = list()
    instrumentalness = list()
    liveness = list()
    valence = list()
    tempo = list()
    duration = list()

    for item in range(0,50):
        danceability.append(feature[item]['danceability'])
        energy.append(feature[item]['energy'])
        loudness.append(feature[item]['loudness'])
        speechiness.append(feature[item]['speechiness'])
        acousticness.append(feature[item]['acousticness'])
        instrumentalness.append(feature[item]['instrumentalness'])
        liveness.append(feature[item]['liveness'])
        valence.append(feature[item]['valence'])
        tempo.append(feature[item]['tempo'])
        duration.append(feature[item]['duration_ms'])
        
        
# dataframe
        
    df = pd.DataFrame(list(zip(track, track_id, artist, album, popularity, danceability,
                              energy,loudness,speechiness,acousticness,instrumentalness,
                              liveness,valence,tempo)),
                    columns = ['Track', 'Track ID', 'Artist', 'Album', 'Rating','Danceability',
                               'Energy','Loudness','Speechiness','Acousticness',
                               'Instrumentalness','Liveness','Valence','Tempo'])
    
    return df